# Multiplicative Inverse

In this example we are once again starting with randomly selected field elements and revealing them. We do not work with encoding or decoding the values in this case since the multiplicative inverse we are dealing with in this context is that associated speicfically with the field, and not relative to the encoded or decoded values we are using as a symatic structure for our fixed point arithemtic. 

This returns an exact multiplicative inverse WRT the field, which is not the same as division, which produces an approximate result.

After producing and revealing the shared secrets we calculate their multiplicative inverse on an elementwise basis in the field, and reveal it. Finally we perform the multiplication and reveal the product thus demonstrating that the correct multiplicative inverses have been calculated since the product is a matrix comprised entirely of 1s. 

In [1]:
import logging

import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)
    generator = numpy.random.default_rng()
    secret_share = protocol.uniform(shape=(2,2))
    secret = protocol.reveal(secret_share)
    log.info(f"Player {communicator.rank} secret: \n{secret}")
    secret_share_inv = protocol.multiplicative_inverse(secret_share)
    secret_inv = protocol.reveal(secret_share_inv)
    log.info(f"Player {communicator.rank} secret inv: \n{secret_inv}")
    product_share = protocol.untruncated_multiply(secret_share, secret_share_inv)
    product = protocol.reveal(product_share)
    log.info(f"Player {communicator.rank} product: \n{product}")


main();

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:58342 from tcp://127.0.0.1:58342.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:58342 from tcp://127.0.0.1:58343.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:58342 from tcp://127.0.0.1:58344.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:root:Player 0 secret: 
[[5017340156550848250 12025498451546507130]
 [13261020176102957045 5794962414364593782]]
INFO:root:Player 1 secret: 
[[5017340156550848250 12025498451546507130]
 [13261020176102957045 5794962414364593782]]
INFO:root:Player 2 secret: 
[[5017340156550848250 12025498451546507130]
 [13261020176102957045 5794962414364593782]]
INFO:root:Player 0 secret inv: 
[[13662618277276158713 13732719500715301478]
 [1518130773047831499 5839521523929247574]]
INF